In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pyarrow as pa
import pyarrow.compute as pc
from nn_rag import Knowledge

### Instantiate capability

In [3]:
kn = Knowledge.from_memory()

In [4]:
tbl = kn.set_source_uri("source/Gen AI Best Practices.pdf").load_source_canonical()
kn.set_persist_uri('./hadron/data/gen_ai_best_practice.parquet')

In [5]:
tbl.shape
tbl.column_names
tbl.column('text').to_pylist()[0][:300]

(1, 1)

['text']

'Best Practices in Generative AI\nResponsible use and development \nin the modern workplace\n© Responsible AI Institute 2024  \nAll Rights Reserved | Do Not Use Without Permission\n\x0cExecutive Summary\nGenerative AI, a technology capable of producing realistic content in the form of text, images,\nsound, and'

### Tidy the text

In [6]:
tbl = kn.tools.pattern_replace(tbl, 'text', '\n', ' ')
tbl = kn.tools.pattern_replace(tbl, 'text', '  ', ' ')

### Profling
#### discovery

In [8]:
sentences = kn.tools.text_profiler(tbl)
sentences.shape
kn.table_report(sentences, head=5)

(392, 5)

,sentence,sentence_num,char_count,word_count,token_count
0,"Best Practices in Generative AI Responsible use and development in the modern workplace © Responsible AI Institute 2024 All Rights Reserved | Do Not Use Without Permission Executive Summary Generative AI, a technology capable of producing realistic content in the form of text, images, sound, and more, presents signiﬁcant opportunities and challenges for businesses today.",0,375,56,94
1,"With generative AI (GenAI) applications ranging from customer service automation to content creation, the recent explosive adoption of LLM technologies like ChatGPT underscores the potential transformative scale of AI impact, both positive and negative.",1,253,34,63
2,"Potential risks and harms from generative AI impact human rights, privacy, security, labor, fairness, sustainability, and more.",2,127,17,32
3,"Without investing effort to comprehensively address these issues across the enterprise, businesses are exposed to the risks of compliance penalties, consumer harm, loss of trust, damages, and more.",3,197,28,49
4,"To position themselves to responsibly capitalize on this potential, organizations must implement governance to pave the way for trustworthy AI deployment, procurement, sale, and use, as applicable.",4,197,27,49


In [9]:
print(f"min word count {pc.min(sentences['word_count']).as_py()}")
print(f"max word count {pc.max(sentences['word_count']).as_py()}")
print(f"mean word count {round(pc.mean(sentences['word_count']).as_py())}")

min word count 1
max word count 188
mean word count 25


In [10]:
print(f"sentence with 2 or less words {pc.count(pc.filter(sentences['word_count'], pc.less_equal(sentences['word_count'], 2))).as_py()}")

sentence with 2 or less words 16


#### remove unwanted sentences

In [13]:
short = kn.tools.filter_on_condition(sentences, header='word_count', condition=(2, 'less', None))

TypeError: Index must either be string or integer

### Chunking

In [ ]:
chunks = kn.tools.text_chunk(sentences, num_sentence_chunk_size=10)
chunks.shape
chunks.column_names

#### ensure paragraphs are properly formed

In [ ]:
# ensure 
chunks = kn.tools.pattern_replace(chunks, '', r'\.([A-Z])', r'. \1', is_regex=True)

### Embedding

In [ ]:
embedding = kn.tools.chunk_embedding(chunks, batch_size=32, embedding_name='all-mpnet-base-v2', device='cpu')

In [ ]:
type(embedding)
embedding.shape

In [ ]:
kn.run_component_pipeline()

### Save Embedding

In [ ]:
kn.save_persist_canonical(embedding)